<a href="https://colab.research.google.com/github/yuriborg/dsmkt/blob/main/churn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações

In [ ]:
pip install google-auth google_auth_oauthlib google-api-python-client

# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys
import time
import datetime
import statistics
import re
import scipy
import unicodedata
import json

import plotly.express as px
from IPython.display import HTML
import plotly.figure_factory as ff

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.rcParams['axes.facecolor'] = 'white'
dt = datetime.datetime.now().strftime('%d%m%Y_%H%M%S')

# Conexão com Sheets

In [ ]:
%python
'''
Conecta no serviço da Google
'''

from __future__ import print_function

import os.path

import numpy as np
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = "/dbfs/FileStore/shared_uploads/rodrigo.sartori@samisaude.com/samisaude_ops_d65992b26dd4.json" #Service account -- Gabriel

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('sheets', 'v4', credentials=creds)


# ---------------------------------------------------------------

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

def read_sheet(
    sheet_ID,
    sheet_range_name
):
    # Call the Sheets API
    fromSheet = service.spreadsheets()
    result_sheet = fromSheet.values().get(spreadsheetId=sheet_ID,
                                range=sheet_range_name).execute()
    values_sheet = result_sheet.get('values', [])
    data = pd.DataFrame(values_sheet[1:], columns=values_sheet[0])
    return data

# Data Engineering Functions

In [ ]:
def apply_standardization(
    data
):
    #Apply functions to remove accents, parentheses with empty spaces
    #replaces upper case letters withll case letters,
    #and replaces empty spaces with underscores
    data.columns = list(map(remove_accents, data.columns))
    data.columns = list(map(lambda x: x.lower().replace(' ', '_').replace('(', '').replace(')', ''),
                             data.columns))
    return data



#Dados dos Sheets

In [ ]:
df_gympass = read_sheet(
    sheet_ID='1p3nqTNrAozYAThAUx9Gt8zWf0E6VGt2R8DSyZkaHaDI',
    sheet_range_name='gympass_projeto_ia_churn.csv!B:F'
)

# remoção de acentos das colunas
df_gympass.columns = list(map(remove_accents, df_gympass.columns))

# padronização
df_gympass.columns = list(map(lambda x: x.lower().replace(' ', '_').replace('(', '').replace(')', ''),
                             df_gympass.columns))

spark.createDataFrame(df_gympass).createOrReplaceTempView("gympass")

In [ ]:
%sql
select
  numero_da_carteirinha as n_carteirinha
  ,count(data_de_utilizacao_gympass_date) as qtd_dias
  ,sum(contagens_de_utilizadores_gympass) as qtd_utilizadores
from gympass
group by numero_da_carteirinha
order by 1
limit 10

In [ ]:
def adjust_retorno_classe(x):
    if x != x or x in [' ', '']:
        return np.nan
    else:
        return x
def adjust_retorno_cpf(x):
    if x != x or x in ['', ' ', '0']:
        return np.nan
    else:
        return int(x)
def adjust_retorno_nota(x):
    if x != x or x in ['', ' ']:
        return np.nan
    else:
        return int(x)

df_retorno = read_sheet(
    sheet_ID='1CUGGkU1zlbNCzxlS2c5LsbVGtuiMyVIm89EDnGnoQso',
    sheet_range_name='NPS_full!A:AE'
)

# remoção de acentos das colunas
df_retorno.columns = list(map(remove_accents, df_retorno.columns))

# padronização
df_retorno.columns = list(map(lambda x: x.lower().replace(' ', '_').replace('(', '').replace(')', ''),
                             df_retorno.columns))

# adjusting "class"
df_retorno = df_retorno.rename(columns = {'class': 'classe'})
df_retorno['classe'] = list(map(adjust_retorno_classe, df_retorno['classe'].values))

# adjusting CPF
df_retorno['CPF'] = list(map(adjust_retorno_cpf, df_retorno['cpf'].values))

# adjusting nota
df_retorno['nota'] = list(map(adjust_retorno_nota, df_retorno.nota.values))

spark.createDataFrame(df_retorno).createOrReplaceTempView("retorno_nps")

display(df_retorno)

In [ ]:


  # selected categorical variables
cat_cols = [
    'tipo_do_contato',
    'nome_pesquisa',
    'pesquisa',
    'classe',
]
# mode function for categorical values
mode_func = lambda x: scipy.stats.mode(list(filter(lambda x_j: x_j is not None, x)) )[0]
d_agg = dict(zip(
    cat_cols,
    [mode_func]*len(cat_cols)
))
d_agg.update({'nota': 'mean'})

# filtering for members and possible members
tipos_de_contato_sel = [
    'Membro - Cancelado',
    'Posvendas',
    'Aguarde',
    'Membro - Inadimplente',
    'Demais_assuntos',
    'Contratante - Não membro',
    '3_Posvendas_Beneficiario',
    'Membro - Aguardando vigência',
    '2_Posvendas_Contratante'
]
# aggregating per member
df_retorno_agg = df_retorno[
    (df_retorno.tipo_do_contato.isin(tipos_de_contato_sel)) & (~df_retorno.cpf.isna())
          ].groupby(['cpf'], as_index = False).agg(d_agg)
# save view
spark.createDataFrame(df_retorno_agg).createOrReplaceTempView("retorno_nps_agg")

df_retorno_agg

In [ ]:
%sql
select
  *
from retorno_nps_agg
limit 10

In [ ]:
%sql
select
  retorno_nps_agg.*
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join retorno_nps_agg on
  retorno_nps_agg.cpf = membros.identity_number
where cpf is not null
    and cpf in (32495031854, 42018098896, 446451398, 32509947826)
--   and count(cpf)
order by cpf
-- limit 30

In [ ]:
%sql
select
    retorno_nps_agg.cpf
    ,avg(retorno_nps_agg.nota) as nota_media
    ,max(retorno_nps_agg.nota) as nota_max
    ,min(retorno_nps_agg.nota) as nota_min
    ,sum(
      case when retorno_nps_agg.nota >= 7 then 1
          else 0 end
    ) as qtd_notas_boas
    ,sum(
      case when retorno_nps_agg.nota > 5 and  retorno_nps_agg.nota < 7 then 1
          else 0 end
    ) as qtd_notas_medias
    ,sum(
      case when retorno_nps_agg.nota <= 5 then 1
          else 0 end
    ) as qtd_notas_ruins
    , last(tipo_do_contato) as tipo_do_contato

from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join retorno_nps_agg on
  retorno_nps_agg.cpf = membros.identity_number
where retorno_nps_agg.cpf is not null
group by retorno_nps_agg.cpf
-- order by count(distinct retorno_nps_agg.tipo_do_contato), count(retorno_nps_agg.cpf) desc

In [ ]:
%sql
select
    retorno_nps_agg.cpf
    ,avg(retorno_nps_agg.nota) as nota_media
    ,max(retorno_nps_agg.nota) as nota_max
    ,min(retorno_nps_agg.nota) as nota_min
    ,sum(
      case when retorno_nps_agg.nota >= 7 then 1
          else 0 end
    ) as qtd_notas_boas
    ,sum(
      case when retorno_nps_agg.nota > 5 and  retorno_nps_agg.nota < 7 then 1
          else 0 end
    ) as qtd_notas_medias
    ,sum(
      case when retorno_nps_agg.nota <= 5 then 1
          else 0 end
    ) as qtd_notas_ruins
    , last(tipo_do_contato) as tipo_do_contato

from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join retorno_nps_agg on
  retorno_nps_agg.cpf = membros.identity_number
where retorno_nps_agg.cpf is not null
group by retorno_nps_agg.cpf
-- order by count(distinct retorno_nps_agg.tipo_do_contato), count(retorno_nps_agg.cpf) desc

In [ ]:
def clean_t_atend(x):
    if x != x or x in ['',' ']:
        return np.nan
    else:
        return float(x.replace(',','.'))

df_tickets = read_sheet(
    sheet_ID='1GDbbvoa8JVTTPuG6DpNLPrvowTThpfwiL5g0R-wePOE',
    sheet_range_name='Tickets!A:BE'
)
# filtro por ID_cont vazio
df_tickets = df_tickets[~df_tickets.ID_cont.isin(['', ' ', np.nan])]

# filtro de suporte a membros
df_tickets = df_tickets[df_tickets.pipeline == 'Suporte a membros']

# remoção de acentos das colunas
df_tickets.columns = list(map(remove_accents, df_tickets.columns))

# padronização
df_tickets.columns = list(map(lambda x: x.lower().replace(' ', '_').replace('(', '').replace(')', ''),
                             df_tickets.columns))

cols = [
    'id',
    't_atend_hora',
    't_atend_s',
#     'nota',
#     't_espera_min',
#     't_espera_hora',
    'categoria',
    'tiquete',
    'canal',
    'canal_critico',
    'id_cont'
]
df_tickets = df_tickets[cols]

# adjust missing, and numerical
df_tickets['t_atend_hora'] = list(map(clean_t_atend, df_tickets['t_atend_hora'].values))
df_tickets['t_atend_s'] = list(map(clean_t_atend, df_tickets['t_atend_s'].values))

spark.createDataFrame(df_tickets).createOrReplaceTempView("tickets")
display(df_tickets)

In [ ]:
# identifying churn-related content
df_tickets['relacionado_a_cancelamento'] = list(map(lambda x: int('cancela' in x.lower()), df_tickets.tiquete.values))

# mode function for categorical values
mode_func = lambda x: scipy.stats.mode(list(filter(lambda x_j: x_j is not None, x)) )[0]
d_agg = {}
num_cols = ['t_atend_hora', 't_atend_s']
for col in list(set(df_tickets.columns) - set(num_cols)):
    d_agg.update({col: mode_func})
for col in num_cols:
    d_agg.update({col: 'mean'})

# aggregating per member
df_tickets_agg = df_tickets[
     (~df_tickets.id.isna())
          ].groupby(['id_cont'], as_index = False).agg(d_agg)

spark.createDataFrame(df_tickets_agg).createOrReplaceTempView("tickets_agg")
df_tickets_agg

In [ ]:
FROM_SPREADSHEET_ID = '1mz7iCptoxYmb8xjFHfa7-vBipB5Hw5McxI1nLSyLwM4'

FROM_RANGE_NAME = 'Página1!A:Q'

# Call the Sheets API
fromSheet = service.spreadsheets()
result_contatoshs = fromSheet.values().get(spreadsheetId=FROM_SPREADSHEET_ID,
                                range=FROM_RANGE_NAME).execute()
values_contatoshs = result_contatoshs.get('values', [])
df_contatoshs = pd.DataFrame(values_contatoshs[1:], columns=values_contatoshs[0])

# padronização
df_contatoshs = apply_standardization(df_contatoshs)
display(df_contatoshs)
spark.createDataFrame(df_contatoshs).createOrReplaceTempView("contatos_hs")


In [ ]:
FROM_SPREADSHEET_ID = '1Ome3zezeWRZBXiY80bagz_9hra-2gelA8YMf1zvhPtk'

FROM_RANGE_NAME = 'CANCELADOS_INAD_METODO_PAGAMENTO!A:I'

# Call the Sheets API
fromSheet = service.spreadsheets()
result_relatorio81_cancel = fromSheet.values().get(spreadsheetId=FROM_SPREADSHEET_ID,
                                range=FROM_RANGE_NAME).execute()
values_relatorio81_cancel = result_relatorio81_cancel.get('values', [])
df_relatorio81_cancel = pd.DataFrame(values_relatorio81_cancel[1:], columns=values_relatorio81_cancel[0])

df_relatorio81_cancel = apply_standardization(df_relatorio81_cancel)
spark.createDataFrame(df_relatorio81_cancel).createOrReplaceTempView("relatorio81_cancel")

display(df_relatorio81_cancel)

In [ ]:
df_relatorio81_cancel[
    df_relatorio81_cancel.cd_contrato == list(filter(lambda cd_contrato: len(df_relatorio81_cancel[df_relatorio81_cancel.cd_contrato == cd_contrato])>1,
               df_relatorio81_cancel.cd_contrato.unique()
               ))[1]
]

In [ ]:
contratos_multiplos_registros = list(filter(lambda cd_contrato: len(df_relatorio81_cancel[df_relatorio81_cancel.cd_contrato == cd_contrato])>1,
               df_relatorio81_cancel.cd_contrato.unique()
               ))

df_relatorio81_cancel[
    df_relatorio81_cancel.cd_contrato.isin(contratos_multiplos_registros)
].sort_values(by=['cd_contrato']).drop_duplicates()

In [ ]:
"""
Acesso a planilha com informações de canal de entrada do Lead.
Extração dos dados: 2022-Set-20.
"""
FROM_SPREADSHEET_ID_Canal = '1l15fU6VbxJqoSZ4fOAxKOLQFTHsIEojQYfeyKlh77B8'
FROM_RANGE_NAME_Canal = 'Numero de Vidas V2!A:D'
# Call the Sheets API
result_Canal = fromSheet.values().get(spreadsheetId=FROM_SPREADSHEET_ID_Canal,
                                   range=FROM_RANGE_NAME_Canal).execute()
values_Canal = result_Canal.get('values', [])
#Crio dataframe com as colunas de interesse. O nome das colunas é definido por values[0].
dfCanal = pd.DataFrame(values_Canal[1:], columns=values_Canal[0])
dfCanal = dfCanal[(dfCanal['Origem do lead']!='') | (dfCanal['Nº do CNPJ']!='')]
dfCanal = dfCanal.rename(columns={'Create Date':'DT_CREATE', '[B2C] Data da fase Pagamento Agendado':'DT_PGTO_AGENDADO', 'Nº do CNPJ':'CNPJ'})
dfCanal = apply_standardization(dfCanal)
spark.createDataFrame(dfCanal).createOrReplaceTempView("aquisicao")
dfCanal

In [ ]:
%sql
select distinct
  membros.contract_id
  ,aquisicao.origem_do_lead
  ,aquisicao.dt_pgto_agendado
  ,aquisicao.dt_create

from hive_metastore.samidp_prd_refined_pme_portal_pme.companies c
inner join aquisicao on
  aquisicao.cnpj = c.document
inner join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
  membros.contract_id = c.customerid
order by 1

In [ ]:
%sql
with aquisicao as (
  select distinct
    membros.contract_id
    ,aquisicao.origem_do_lead
    ,aquisicao.dt_pgto_agendado
    ,aquisicao.dt_create

  from hive_metastore.samidp_prd_refined_pme_portal_pme.companies c
  inner join aquisicao on
    aquisicao.cnpj = c.document
  inner join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
    membros.contract_id = c.customerid
  order by 1
)
select distinct
  aquisicao.contract_id
  ,aquisicao.origem_do_lead
  ,aquisicao.dt_pgto_agendado
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join aquisicao on
  aquisicao.contract_id = membros.contract_id
where aquisicao.contract_id is not null
order by 1

# Dados do banco(sandbox)

In [ ]:
%sql
select
*
from hive_metastore.samidp_prd_refined_oracle_dbaps.carencia_usuario carencia
      left join hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
        on carencia.cd_matricula = guia.cd_matricula

In [ ]:
%sql
select * from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
where ds_cirurgia is not null

In [ ]:
%sql
select distinct
    cd_matricula
    ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
    ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  where dt_autorizacao is not null
  order by 1,2

In [ ]:
%sql
select distinct
    cd_matricula
--     ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
    ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
--     ,dt_autorizacao
  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  where dt_autorizacao is not null
  order by 1,2

In [ ]:
%sql
select
--     distinct
    cd_matricula
    ,dt_autorizacao
--     ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
--     ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
    ,count(*) over (partition by cd_matricula) as qtd_guias
  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  where dt_autorizacao is not null
    and cd_matricula = 60097
  order by 1,2

In [ ]:
%sql
with uso_mensal as (
  select distinct
    cd_matricula
    ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
    ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
    ,count(*) over (partition by cd_matricula) as qtd_guias
  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  where dt_autorizacao is not null
  order by 1,2
)
select distinct
  uso_mensal.cd_matricula
  ,avg(uso_mensal.qtd_guias_mes) as media_guias_por_mes
  ,avg(uso_mensal.qtd_guias) as soma_qtd_guias
  ,count(distinct uso_mensal.anomes) as qtd_meses
--   ,avg(uso_mensal.qtd_guias) / count(distinct uso_mensal.anomes) as razao_qtd_guias_qtd_meses

from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
left join uso_mensal on
  uso_mensal.cd_matricula = guia.cd_matricula

where dt_autorizacao is not null
group by uso_mensal.cd_matricula
order by 1

In [ ]:
%sql
select distinct
    cd_matricula
    ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
    ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
    ,count(*) over (partition by cd_matricula) as qtd_guias
    ,case when dt_autorizacao is not null ---and ds_senha_autorizacao is not null
--               then 'autorizada'
--           else 'nao_autorizada'
            then 1 else 0
          end as autorizacao
    ,dt_autorizacao
    ,ds_senha_autorizacao

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
--   where dt_autorizacao is not null
  order by 1,2

In [ ]:
%sql
with uso_mensal as (
  select distinct
    cd_matricula
    ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
    ,count(*) over (partition by cd_matricula, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
    ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by cd_matricula) as qtd_meses
    ,count(*) over (partition by cd_matricula) as qtd_guias
    ,case when dt_autorizacao is not null ---and ds_senha_autorizacao is not null
--             then 'autorizada'
--           else 'nao_autorizada'
          then 1 else 0
          end as autorizacao

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  order by 1,2
)
select distinct
  uso_mensal.cd_matricula
  ,avg(uso_mensal.qtd_guias_mes) as media_guias_por_mes
  ,avg(uso_mensal.qtd_guias) as soma_qtd_guias
  ,count(distinct uso_mensal.anomes) as qtd_meses
--   ,avg(uso_mensal.qtd_guias) / count(distinct uso_mensal.anomes) as razao_qtd_guias_qtd_meses
  ,sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes) as qtd_guias_autorizadas
  ,(sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes))/avg(uso_mensal.qtd_guias) as pct_guias_aceitas

from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
left join uso_mensal on
  uso_mensal.cd_matricula = guia.cd_matricula

where dt_autorizacao is not null
group by uso_mensal.cd_matricula
order by 1

In [ ]:
%sql
select distinct
  membros.contract_id
  ,guia.cd_prestador
  ,count(guia.cd_prestador) as freq_prestador

from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
left join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
  guia.nr_carteira_utilizada = membros.health_card_number
where contract_id in (136, 127)
-- where contract_id is not null
group by
  membros.contract_id
  ,guia.cd_prestador
-- order by 1, 3 desc, 2
order by 1, 3 desc

In [ ]:
%sql
with prestadores as (
  select distinct
    membros.contract_id
    ,guia.cd_prestador
    ,count(guia.cd_prestador) as freq_prestador

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  left join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
    guia.nr_carteira_utilizada = membros.health_card_number
  group by
    membros.contract_id
    ,guia.cd_prestador
  order by 3 asc
)
select distinct
  membros.contract_id
  ,first(prestadores.freq_prestador) over (partition by prestadores.contract_id order by prestadores.freq_prestador desc) as freq_prestador_mais_freq
  ,first(prestadores.cd_prestador) over (partition by prestadores.contract_id order by prestadores.freq_prestador desc ) as prestador_mais_freq

from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join prestadores on
  membros.contract_id = prestadores.contract_id
where membros.contract_id in (479, 480, 647, 136, 127) ---(136, 127)
order by 1

In [ ]:
%sql
with prestadores as (
  select distinct
    membros.contract_id
    ,guia.cd_prestador
    ,count(guia.cd_prestador) as freq_prestador

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  left join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
    guia.nr_carteira_utilizada = membros.health_card_number
  group by
    membros.contract_id
    ,guia.cd_prestador
  order by 3 desc
)
select distinct
  prestadores.contract_id
  ,first(prestadores.freq_prestador)  as freq_prestador_mais_freq
  ,first(prestadores.cd_prestador) as prestador_mais_freq

from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join prestadores on
  membros.contract_id = prestadores.contract_id
where prestadoresmembros.contract_id in (479, 480, 647, 136, 127)
group by prestadores.contract_id
-- order by 1

In [ ]:
%sql
select
--     guia.*
    carencia.cd_matricula
    ,carencia.cd_carencia_usuario
    ,carencia.cd_gru_carencia
    ,max(carencia.dt_inicio_carencia) as dt_max_carencia
    ,min(carencia.dt_inicio_carencia) as dt_min_carencia
    ,max(carencia.dt_inclusao) as dt_max_inclusao
    ,min(carencia.dt_inclusao) as dt_min_inclusao
    ,max(carencia.dt_vencimento) as dt_max_vencimento
    ,min(carencia.dt_vencimento) as dt_min_vencmento
    ,sum(carencia.nr_dias_aprov_carencia) as total_dias_aprov_carencia
--     ,carencia.dt_inicio_carencia
--     ,carencia.dt_inclusao

  from hive_metastore.samidp_prd_refined_oracle_dbaps.carencia_usuario carencia
  left join hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia on
    guia.cd_matricula = carencia.cd_matricula

  group by carencia.cd_matricula, carencia.cd_carencia_usuario, carencia.cd_gru_carencia

In [ ]:
%sql
with carencias as (
  select
    guia.nr_carteira_utilizada
    ,carencia.*
  from hive_metastore.samidp_prd_refined_oracle_dbaps.carencia_usuario carencia
  left join hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia on
    guia.cd_matricula = carencia.cd_matricula
)
select
  membros.contract_id
  ,membros.member_id

--   ,carencias.cd_matricula
  ,carencias.cd_carencia_usuario
  ,carencias.cd_gru_carencia
  ,max(carencias.dt_inicio_carencia) as dt_max_carencia
  ,min(carencias.dt_inicio_carencia) as dt_min_carencia
  ,max(carencias.dt_inclusao) as dt_max_inclusao
  ,min(carencias.dt_inclusao) as dt_min_inclusao
  ,max(carencias.dt_vencimento) as dt_max_vencimento
  ,min(carencias.dt_vencimento) as dt_min_vencmento
  ,sum(carencias.nr_dias_aprov_carencia) as total_dias_aprov_carencia

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  left join carencias on
    carencias.nr_carteira_utilizada = membros.health_card_number

  group by membros.contract_id
           ,membros.member_id
--            ,carencias.cd_matricula
           ,carencias.cd_carencia_usuario
           ,carencias.cd_gru_carencia

In [ ]:
%sql
select
*
from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato
where dt_reativacao is null
limit 5

In [ ]:
%sql
with desligamento as (
  select
    cd_contrato
    ,1 as desligado
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
  where dt_reativacao is null
)
select
  membros.contract_id
  ,COALESCE(desligamento.desligado, 0) AS e_desligado
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join desligamento on
  desligamento.cd_contrato = membros.contract_id
limit 10


In [ ]:
query="""with desligamento as (
  select
    cd_contrato
    ,1 as desligado
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
)
select
  membros.contract_id
  ,COALESCE(desligamento.desligado, 0) AS e_desligado
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join desligamento on
  desligamento.cd_contrato = membros.contract_id"""

t = spark.sql(query).toPandas()
pd.DataFrame({
    'classe': list(t['e_desligado'].value_counts().index),
    'freq': t['e_desligado'].value_counts().values,
    '%': (t['e_desligado'].value_counts()*100/len(t)).values
})

In [ ]:

%sql
with desligamento as (
  select
    cd_contrato
    ,1 as desligado
    ,dt_desligamento
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
  where dt_reativacao is null
)
select
  distinct
  membros.contract_id
  ,COALESCE(desligamento.desligado, 0) AS e_desligado
  ,datediff(desligamento.dt_desligamento, membros.member_entrance_date) as tempo_contrato
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join desligamento on
  desligamento.cd_contrato = membros.contract_id
limit 10


In [ ]:
%sql
with desligamento as (
  select
    cd_contrato
    ,1 as desligado
    ,dt_desligamento
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
)
select
  distinct
--   membros.contract_id
--   ,COALESCE(desligamento.desligado, 0) AS e_desligado
  datediff(desligamento.dt_desligamento, membros.member_entrance_date)/30 as tempo_churn_meses
  ,membros.member_entrance_date
  ,desligamento.dt_desligamento
from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
left join desligamento on
  desligamento.cd_contrato = membros.contract_id
where datediff(desligamento.dt_desligamento, membros.member_entrance_date) is not null
limit 10


In [ ]:
def extract_json_answers(data):
    # getting the keys
    ks = list(json.loads(data['answers'].values[0])[0].keys())

    for k in ks: # for each dict key
        data[f'answers_{k}'] = list(map(
                                    lambda x: json.loads(x)[0][k], # extract json for the key k
                                        data['answers'].values
                                ))
    data = data.drop(['answers', 'surveyresults'], axis = 1)
    return data


def create_answers_view_suasaude():
    t = spark.sql("select * from hive_metastore.samidp_prd_refined_app_survey.answers where surveytitle = 'Sua Saude'")
    t = t.toPandas()
    t = extract_json_answers(data = t)

    # filtering relevant columns
    t = t[[
        'id',
        'responsedby',
        'completeddate',
        'status',
        'answers_value',
        'answers_weigth',
        'answers_optionId',
        'answers_questionTitle',
    ]]
    # saving as view
    spark.createDataFrame(t).createOrReplaceTempView("declaracao_saude")


create_answers_view_suasaude()

In [ ]:
def extract_answers_pesquisaHRA(t):
    # selected columns
    columns = [
        'id',
        'responsedby',
        'completeddate',
        'status'
    ]
    data = pd.DataFrame()
    for i in range(len(t)):

        answers = list(map(
                lambda x: json.loads(x),
                    t.loc[[i]]['answers'].values
            ))[0]

        # concatenating answers
        df_ans_i = pd.DataFrame()
        for ans in answers:
            df_ans_i = pd.concat([
                            df_ans_i,
                            pd.DataFrame(ans, index=[0])
                        ], axis =0 )
        for col in columns: # getting the other columns and values
            df_ans_i[col] = len(df_ans_i)*[t.loc[[i]][col].values[0]]

        # concatenating for each index
        data = pd.concat([
            data,
            df_ans_i
        ], axis=0)

    # renaming the answers-keys columns
    ks = ['value', 'weigth', 'optionId', 'questionId', 'questionType', 'questionTitle']
    data = data.rename(
        dict(zip(
            ks,
            list(map(lambda x: f'answers_{x}', ks))
        )),
        axis = 'columns'
    )
    # selecting the order of columns
    data = data[
        columns + list(set(data.columns) - set(columns))
    ]
    return data.reset_index(drop=True)

def create_answers_view_HRA():
    t = spark.sql("select * from hive_metastore.samidp_prd_refined_app_survey.answers where surveytitle = 'Pesquisa HRA'")
    t = t.toPandas()
    t = extract_answers_pesquisaHRA(t = t)

    # filtering relevant columns
    t = t[[
        'id',
        'responsedby',
        'completeddate',
        'status',
        'answers_value',
        'answers_weigth',
        'answers_optionId',
        'answers_questionTitle',
    ]]
    # saving as view
    spark.createDataFrame(t).createOrReplaceTempView("pesquisa_hra")

# def transpose_pesquisa_hra(data,
#                           id_col = 'responsedby'
#                           ):
#     questions = [
# #                 'Nas últimas 2 semanas, sentiu:',
# #                 'Tem ou já teve alguma dessas condições?',
#                 'Numa escala de 1 a 10, qual nota você daria para a sua saúde hoje?',
#                 'Faz atividade física regularmente?',
# #                 'Qual o seu peso?',
# #                 'Qual a sua altura?',
#                 'Você fuma?'
#                 ]
#     t = pd.DataFrame()
#     for q in questions:
#         temp = pd.DataFrame()
#         temp[q] = data[data.answers_questionTitle==q]['answers_value'].values
#         temp[id_col] = data[data.answers_questionTitle==q][id_col].values

#         t = pd.concat([t, temp], axis =0)

#     return t[[id_col] + list(set(t.columns) - set([id_col]))]


def transpose_pesquisa_hra(data,
                          verbose = False
                          ):
    cpfs = list(data.responsedby.unique())

    questions = [
                'Numa escala de 1 a 10, qual nota você daria para a sua saúde hoje?',
                'Faz atividade física regularmente?',
                'Você fuma?'
                ]
    d = dict(zip(questions, [[] for _ in range(len(questions))]))
    d.update({'responsedby': cpfs})

    N = len(cpfs)

    for i, x in enumerate(cpfs):
        if verbose:
            print('({}/{}) ({:.3f}%)'.format(i+1, N, (i+1)*100/N))
        for q in questions:
            v = data[(data.responsedby == x) & (data.answers_questionTitle == q)].answers_value.values
            if len(v)>0:
                d[q].append(v[0])
            else:
                d[q].append(np.nan)
    res = pd.DataFrame(d)
    res.columns = list(map(lambda x: remove_accents(x).lower().replace('?','').replace(' ','_').replace(',',''), res.columns))
    return res


def create_hra_agg_view(
    create_temp_view = True
    ):
    if create_temp_view:
        create_answers_view_HRA()

    t = spark.sql('select * from pesquisa_hra')
    t = t.toPandas()
    t_agg = transpose_pesquisa_hra(t)

    t_agg = apply_standardization(t_agg)
    t_agg.columns = list(map(lambda x: x.lower().replace('?', '').replace(',',''),
                             t_agg.columns))
    spark.createDataFrame(t_agg).createOrReplaceTempView("pesquisa_hra_agg")

create_hra_agg_view()

In [ ]:
t = spark.sql('select * from pesquisa_hra')
t = t.toPandas()

def adjust_hra_table(data):
    cpfs = list(data.responsedby.unique())

    questions = [
                'Numa escala de 1 a 10, qual nota você daria para a sua saúde hoje?',
                'Faz atividade física regularmente?',
                'Você fuma?',
                'Qual o seu peso?',
                'Qual a sua altura?'
                ]
    d = dict(zip(questions, [[] for _ in range(len(questions))]))
    d.update({'responsedby': cpfs})

    N = len(cpfs)

    for i, x in enumerate(cpfs):
        print('({}/{}) ({:.3f}%)'.format(i+1, N, (i+1)*100/N))
        for q in questions:
            v = data[(data.responsedby == x) & (data.answers_questionTitle == q)].answers_value.values
            if len(v)>0:
                d[q].append(v[0])
            else:
                d[q].append(np.nan)
    res = pd.DataFrame(d)
    res.columns = list(map(lambda x: remove_accents(x).lower().replace('?','').replace(' ','_').replace(',',''), res.columns))
    return res

t_agg = adjust_hra_table(t)

In [ ]:
def calcula_imc(peso, altura):
    """https://centrodeobesidadeediabetes.org.br/tudo-sobre-obesidade/calculadora-de-imc/#:~:text=%C3%89%20calculado%20dividindo%20o%20peso,ao%20quadrado%20(em%20metros)."""
    if peso != peso or altura != altura:
        return np.nan
    else:
        return float(peso)/(float(altura)*float(altura))
def avalia_imc(imc):
    """https://www.uol.com.br/vivabem/noticias/bbc/2018/12/07/imc-qual-e-seu-indice-de-massa-corporal-e-o-que-isso-significa-para-saude.htm#:~:text=De%20acordo%20com%20os%20especialistas,de%2030%20s%C3%A3o%20classificados%20obesos."""
    if imc >= 18.5 and imc < 25:
        return 'regular'
    elif imc >= 25 and imc < 30:
        return 'sobrepeso'
    elif imc >= 30 and imc < 40:
        return 'obeso'
    elif imc >= 40:
        return 'muito obeso'
    else:
        return 'subpeso'

t_agg = t_agg.fillna(value = np.nan)
t_agg['imc'] = list(map(calcula_imc,
                       t_agg['qual_o_seu_peso'].values,
                       t_agg['qual_a_sua_altura'].values
                       ))
t_agg['avaliacao_imc'] = list(map(avalia_imc, t_agg['imc'].values))
t_agg

In [ ]:
%sql
WITH INFOS_MEMBROS AS (
  select distinct

    membros.contract_id
    ,membros.member_id
    ,membros.product_id
    ,membros.company_id
    ,membros.health_card_number
    ,membros.identity_number
    ,titular_id

    --- infos gerais
--     ,datediff(getdate(), membros.member_entrance_date) as tempo_de_casa
    ,membros.member_entrance_date
    ,membros.gender
    ,membros.age
    ,membros.status
    ,membros.neighborhood
    ,membros.city
    ,membros.state
    ,membros.dependency_relationship

    --- estatísticas de idades
    ,max(membros.age) over (partition by contract_id) as idade_max
    ,min(membros.age) over (partition by contract_id) as idade_min

    --- dependentes
    ,count(membros.dependency_relationship) over (partition by contract_id) as qtd_dependentes -- se for titular é null

    --- entrada
    ,membros.member_entrance_date

     --- variáveis indicadoras
    ,case when membros.gender = 'M' then 1 else 0 end as genero_mas
    ,case when membros.gender = 'F' then 1 else 0 end as genero_fem

    ,case when membros.age >= 60 then 1 else 0 end as idoso
    ,case when membros.age > 18 and membros.age <= 30 then 1 else 0 end as jovem
    ,case when membros.age > 30 and membros.age < 60 then 1 else 0 end as adulto
    ,case when membros.age <= 18 then 1 else 0 end as crianca


  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  where is_samer = 'nao' -- sem colaboradores
  order by 1
)
,estados_freq as (
  select
    INFOS_MEMBROS.contract_id
    ,INFOS_MEMBROS.state
    , count(INFOS_MEMBROS.state) as freq
  from INFOS_MEMBROS
  group by INFOS_MEMBROS.contract_id
            , INFOS_MEMBROS.state
  order by
      INFOS_MEMBROS.contract_id
      ,count(INFOS_MEMBROS.state)
      desc
)
-- select
--     INFOS_MEMBROS.contract_id
--     ,INFOS_MEMBROS.state
--     , count(INFOS_MEMBROS.state) as freq
--   from INFOS_MEMBROS
--   where INFOS_MEMBROS.contract_id in (479, 480, 647)
--   group by INFOS_MEMBROS.contract_id
--             , INFOS_MEMBROS.state
select

 INFOS_MEMBROS.contract_id
,last(estados_freq.state) as estado_moda
 ,last(estados_freq.freq) as freq
 ,count(distinct estados_freq.state) as qtd_estados
 ,count(estados_freq.state) as soma_freqs
 ,last(estados_freq.freq)/count(estados_freq.state) as pct_estado

from INFOS_MEMBROS
left join estados_freq on
  estados_freq.contract_id = INFOS_MEMBROS.contract_id

where INFOS_MEMBROS.contract_id in (479, 480, 647)
group by INFOS_MEMBROS.contract_id
order by 1

In [ ]:
%sql
select distinct

    membros.contract_id
    ,membros.member_id
    ,membros.product_id
    ,membros.company_id
    ,membros.health_card_number
    ,membros.identity_number
    ,titular_id

    --- infos gerais
--     ,datediff(getdate(), membros.member_entrance_date) as tempo_de_casa

    ,membros.member_entrance_date
    ,membros.gender
    ,membros.age
    ,membros.status
    ,membros.neighborhood
    ,membros.city
    ,membros.state
    ,membros.dependency_relationship

    --- estatísticas de idades
    ,max(membros.age) over (partition by contract_id) as idade_max
    ,min(membros.age) over (partition by contract_id) as idade_min

    --- dependentes
    ,count(membros.dependency_relationship) over (partition by contract_id) as qtd_dependentes -- se for titular é null

    --- entrada
    ,membros.member_entrance_date

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  where is_samer = 'nao' -- sem colaboradores
  order by 1

In [ ]:
%sql

WITH INFOS_MEMBROS AS (
  select distinct

    membros.contract_id
    ,membros.member_id
    ,membros.product_id
    ,membros.company_id
    ,membros.health_card_number
    ,membros.identity_number
    ,titular_id

    --- infos gerais
--     ,datediff(getdate(), membros.member_entrance_date) as tempo_de_casa
    ,membros.member_entrance_date
    ,membros.gender
    ,membros.age
    ,membros.status
    ,membros.neighborhood
    ,membros.city
    ,membros.state
    ,membros.dependency_relationship

    --- estatísticas de idades
    ,max(membros.age) over (partition by contract_id) as idade_max
    ,min(membros.age) over (partition by contract_id) as idade_min

    --- dependentes
    ,count(membros.dependency_relationship) over (partition by contract_id) as qtd_dependentes -- se for titular é null

    --- entrada
    ,membros.member_entrance_date

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  where is_samer = 'nao' -- sem colaboradores
  order by 1
)
,GYMPASS AS (
  select
    numero_da_carteirinha as health_card_number
    ,count(data_de_utilizacao_gympass_date) as qtd_dias_gympass
    ,sum(contagens_de_utilizadores_gympass) as qtd_utilizadores_gympass
  from gympass
  group by numero_da_carteirinha
  order by 1
)
,PERFIL_INAD AS (
  select
    cd_contrato
    ,sum(dias_atraso) as total_dias_atraso
    ,avg(dias_atraso) as media_dias_atraso
    ,sum(vl_mensalidade) as total_vl_mensalidade
    ,avg(vl_mensalidade) as media_vl_mensalidade

    -- motivo

  from relatorio81_inad
  group by cd_contrato
  order by 1
)
,RELATORIO81_CANCELAMENTO AS (
   select distinct
     cd_contrato as cd_contrato,
     max(tempo_de_sami_meses) over (partition by cd_contrato) as tempo_de_sami_meses -- maior tempo de contrato por contrato (tempo de contrato mais atualizado)
  from relatorio81_cancel
  order by 1
)
,TICKETS AS (
  select
    contatos_hs.cpf
    ,tickets_agg.*
  from tickets_agg
  inner join contatos_hs on
  tickets_agg.id_cont = contatos_hs.id
)
,USO_GUIAS as (
  with uso_mensal as (
    select distinct
      nr_carteira_utilizada
      ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
      ,count(*) over (partition by nr_carteira_utilizada, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
      ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by nr_carteira_utilizada) as qtd_meses
      ,count(*) over (partition by nr_carteira_utilizada) as qtd_guias
      ,case when dt_autorizacao is not null ---and ds_senha_autorizacao is not null
          then 1 else 0
          end as autorizacao
    from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
    where dt_autorizacao is not null
    order by 1,2
  )
  select distinct
    uso_mensal.nr_carteira_utilizada
    ,avg(uso_mensal.qtd_guias_mes) as media_guias_por_mes
    ,avg(uso_mensal.qtd_guias) as soma_qtd_guias
    ,count(distinct uso_mensal.anomes) as qtd_meses
--     ,avg(uso_mensal.qtd_guias) / count(distinct uso_mensal.anomes) as razao_qtd_guias_qtd_meses
    ,sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes) as qtd_guias_autorizadas
    ,(sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes))/avg(uso_mensal.qtd_guias) as pct_guias_autorizadas

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  left join uso_mensal on
    uso_mensal.nr_carteira_utilizada = guia.nr_carteira_utilizada

  where dt_autorizacao is not null
  group by uso_mensal.nr_carteira_utilizada
  order by 1
)
,DS as (
  select
      responsedby as cpf
      ,answers_value as decl_saude_nota_para_saude
  from declaracao_saude
)
,HRA as (
  select
    responsedby as cpf
    ,voce_fuma as hra_fuma
    ,faz_atividade_fisica_regularmente as hra_faz_ativ_fisica
    ,numa_escala_de_1_a_10_qual_nota_voce_daria_para_a_sua_saude_hoje as hra_nota_saude
  from pesquisa_hra_agg
)
,desligamento as (
  select
    cd_contrato
    ,1 as desligado
    ,dt_desligamento
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
  where dt_reativacao is null
)
select distinct
  INFOS_MEMBROS.*

  ,COALESCE(desligamento.desligado, 0) AS e_desligado
  ,datediff(desligamento.dt_desligamento, INFOS_MEMBROS.member_entrance_date)/30 as tempo_churn_meses

  ,GYMPASS.qtd_dias_gympass
--   ,GYMPASS.qtd_utilizadores_gympass

  ,retorno_nps_agg.nota as nota_nps
  ,retorno_nps_agg.tipo_do_contato as tp_contato_nps

--   ,tickets_agg.t_atend_hora

  ,PERFIL_INAD.total_dias_atraso
  ,PERFIL_INAD.media_dias_atraso
  ,PERFIL_INAD.total_vl_mensalidade
  ,PERFIL_INAD.media_vl_mensalidade

  ,RELATORIO81_CANCELAMENTO.tempo_de_sami_meses


  ,USO_GUIAS.media_guias_por_mes
  ,USO_GUIAS.soma_qtd_guias
  ,USO_GUIAS.qtd_meses
  ,USO_GUIAS.qtd_guias_autorizadas
  ,USO_GUIAS.pct_guias_autorizadas

  ,DS.decl_saude_nota_para_saude

  ,HRA.hra_faz_ativ_fisica
  ,HRA.hra_fuma
  ,HRA.hra_nota_saude

  from INFOS_MEMBROS

  left join GYMPASS on
  GYMPASS.health_card_number = INFOS_MEMBROS.health_card_number

  left join retorno_nps_agg on
  retorno_nps_agg.cpf = INFOS_MEMBROS.identity_number

  left join PERFIL_INAD on
  PERFIL_INAD.cd_contrato = INFOS_MEMBROS.contract_id

  left join RELATORIO81_CANCELAMENTO on
  RELATORIO81_CANCELAMENTO.cd_contrato = INFOS_MEMBROS.contract_id

  left join tickets on
  tickets.cpf = INFOS_MEMBROS.identity_number
--   inner join tickets_agg one
--   tickets_agg.id = INFOS_MEMBROS.?
--   where contract_id = 131

  left join USO_GUIAS on
    USO_GUIAS.nr_carteira_utilizada = INFOS_MEMBROS.health_card_number

  left join DS on
    DS.cpf = INFOS_MEMBROS.identity_number

  left join HRA on
    HRA.cpf = INFOS_MEMBROS.identity_number

  left join desligamento on
    desligamento.cd_contrato = INFOS_MEMBROS.contract_id

  group by contract_id
  order by contract_id
--     , member_id
--     , product_id
--     , company_id
--     , health_card_number
--     , identity_number

In [ ]:
%sql
WITH INFOS_MEMBROS AS (
  select distinct

    membros.contract_id
    ,membros.member_id
    ,membros.product_id
    ,membros.company_id
    ,membros.health_card_number
    ,membros.identity_number
    ,titular_id

    --- infos gerais
--     ,datediff(getdate(), membros.member_entrance_date) as tempo_de_casa
    ,membros.member_entrance_date
    ,membros.gender
    ,membros.age
    ,membros.status
    ,membros.neighborhood
    ,membros.city
    ,membros.state
    ,membros.dependency_relationship

    --- estatísticas de idades
    ,max(membros.age) over (partition by contract_id) as idade_max
    ,min(membros.age) over (partition by contract_id) as idade_min

    --- dependentes
    ,count(membros.dependency_relationship) over (partition by contract_id) as qtd_dependentes -- se for titular é null

    --- entrada
    ,membros.member_entrance_date

     --- variáveis indicadoras
    ,case when membros.gender = 'M' then 1 else 0 end as genero_mas
    ,case when membros.gender = 'F' then 1 else 0 end as genero_fem

    ,case when membros.age >= 60 then 1 else 0 end as idoso
    ,case when membros.age > 18 and membros.age <= 30 then 1 else 0 end as jovem
    ,case when membros.age > 30 and membros.age < 60 then 1 else 0 end as adulto
    ,case when membros.age <= 18 then 1 else 0 end as crianca


  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  where is_samer = 'nao' -- sem colaboradores
  order by 1
)
,estados_freq as (
  select
    INFOS_MEMBROS.contract_id
    ,INFOS_MEMBROS.member_id
    ,INFOS_MEMBROS.state as estado
    , count(INFOS_MEMBROS.state) as freq
  from INFOS_MEMBROS
  group by
            INFOS_MEMBROS.contract_id
            ,INFOS_MEMBROS.member_id
            , INFOS_MEMBROS.state
  order by
      INFOS_MEMBROS.member_id
      ,count(INFOS_MEMBROS.state)
      desc
)
,RELATORIO81_CANCELAMENTO AS (
   select distinct
     cd_contrato as cd_contrato,
     max(tempo_de_sami_meses) over (partition by cd_contrato) as tempo_de_sami_meses -- maior tempo de contrato por contrato (tempo de contrato mais atualizado)
  from relatorio81_cancel
  order by 1
)
,desligamento as (
  select
    cd_contrato
    ,1 as desligado
    ,dt_desligamento
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
  where dt_reativacao is null
)
,membros_desligados as (
  select
    membros.contract_id
    ,membros.member_id
    ,desligamento.dt_desligamento
    ,COALESCE(desligamento.desligado, 0) AS e_desligado
    ,datediff(desligamento.dt_desligamento, membros.member_entrance_date)/30 as tempo_churn_meses

    ,case when desligamento.dt_desligamento is not null then
              datediff(desligamento.dt_desligamento, membros.member_entrance_date)/30
          when desligamento.dt_desligamento is null and membros.status ='ativo' then
              datediff(CURRENT_TIMESTAMP(), membros.member_entrance_date)/30
          else null
          end as tempo_contrato_meses

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros

  left join desligamento on
    desligamento.cd_contrato = membros.contract_id

--   group by membros.contract_id
  order by 1
)
,GYMPASS AS (
  select
    numero_da_carteirinha as health_card_number
    ,count(data_de_utilizacao_gympass_date) as qtd_dias_gympass
    ,sum(contagens_de_utilizadores_gympass) as qtd_utilizadores_gympass
  from gympass
  group by numero_da_carteirinha
  order by 1
)
,PERFIL_INAD AS (
  select
    cd_contrato
    ,sum(dias_atraso) as total_dias_atraso
    ,avg(dias_atraso) as media_dias_atraso
    ,sum(vl_mensalidade) as total_vl_mensalidade
    ,avg(vl_mensalidade) as media_vl_mensalidade
    -- motivo

  from relatorio81_inad
  group by cd_contrato
  order by 1
)
,TICKETS AS (
  select
    contatos_hs.cpf
    ,tickets_agg.*
  from tickets_agg
  inner join contatos_hs on
  tickets_agg.id_cont = contatos_hs.id
)
, nps_agg as (
  select
      retorno_nps_agg.cpf
      ,avg(retorno_nps_agg.nota) as nota_nps_media
      ,max(retorno_nps_agg.nota) as nota_nps_max
      ,min(retorno_nps_agg.nota) as nota_nps_min
      ,sum(
        case when retorno_nps_agg.nota >= 7 then 1
            else 0 end
      ) as qtd_notas_nps_boas
      ,sum(
        case when retorno_nps_agg.nota > 5 and  retorno_nps_agg.nota < 7 then 1
            else 0 end
      ) as qtd_notas_nps_medias
      ,sum(
        case when retorno_nps_agg.nota <= 5 then 1
            else 0 end
      ) as qtd_notas_nps_ruins
      , last(retorno_nps_agg.tipo_do_contato) as tipo_do_contato_nps
      ,retorno_nps_agg.tipo_do_contato
      ,count(retorno_nps_agg.tipo_do_contato) as freq_tipo_do_contato_nps
  from retorno_nps_agg
  where retorno_nps_agg.cpf is not null
  group by retorno_nps_agg.cpf
          , retorno_nps_agg.tipo_do_contato
  order by retorno_nps_agg.cpf
          , retorno_nps_agg.tipo_do_contato desc
)
,USO_GUIAS as (
  with uso_mensal as (
    select distinct
      nr_carteira_utilizada
      ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
      ,count(*) over (partition by nr_carteira_utilizada, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
      ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by nr_carteira_utilizada) as qtd_meses
      ,count(*) over (partition by nr_carteira_utilizada) as qtd_guias
      ,case when dt_autorizacao is not null ---and ds_senha_autorizacao is not null
          then 1 else 0
          end as autorizacao
    from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
    where dt_autorizacao is not null
    order by 1,2
  )
  select distinct
    uso_mensal.nr_carteira_utilizada
    ,avg(uso_mensal.qtd_guias_mes) as media_guias_por_mes
    ,avg(uso_mensal.qtd_guias) as soma_qtd_guias
    ,count(distinct uso_mensal.anomes) as qtd_meses
--     ,avg(uso_mensal.qtd_guias) / count(distinct uso_mensal.anomes) as razao_qtd_guias_qtd_meses
    ,sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes) as qtd_guias_autorizadas
    ,(sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes))/avg(uso_mensal.qtd_guias) as pct_guias_autorizadas

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  left join uso_mensal on
    uso_mensal.nr_carteira_utilizada = guia.nr_carteira_utilizada

  where dt_autorizacao is not null
  group by uso_mensal.nr_carteira_utilizada
  order by 1
)
,prestadores as (
  with prestadores_freq as (
    select distinct
      membros.contract_id
      ,guia.cd_prestador
      ,count(guia.cd_prestador) as freq_prestador

    from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
    left join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
      guia.nr_carteira_utilizada = membros.health_card_number
    group by
      membros.contract_id
      ,guia.cd_prestador
    order by 3 asc
  )
  select distinct
    membros.contract_id
    ,first(prestadores_freq.freq_prestador) over (
          partition by prestadores_freq.contract_id order by prestadores_freq.freq_prestador desc
          ) as freq_prestador_mais_freq
    ,first(prestadores_freq.cd_prestador) over (
          partition by prestadores_freq.contract_id order by prestadores_freq.freq_prestador desc
          ) as prestador_mais_freq

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  left join prestadores_freq on
    membros.contract_id = prestadores_freq.contract_id
  order by 1
)
,DS as (
  select
      responsedby as cpf
      ,answers_value as decl_saude_nota_para_saude
  from declaracao_saude
)
,aquisicao as (
  select distinct
    membros.contract_id
    ,aquisicao.origem_do_lead
    ,aquisicao.dt_pgto_agendado
    ,aquisicao.dt_create

  from hive_metastore.samidp_prd_refined_pme_portal_pme.companies c
  inner join aquisicao on
    aquisicao.cnpj = c.document
  inner join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
    membros.contract_id = c.customerid
  order by membros.contract_id, aquisicao.dt_pgto_agendado
)
select
--     distinct
    INFOS_MEMBROS.contract_id
    ,count(INFOS_MEMBROS.member_id) as qtd_membros

    ---------------- dados de cancelamento
   ,last(e_desligado) AS e_desligado
   ,last(tempo_churn_meses) as tempo_churn_meses
    ,last(tempo_contrato_meses) as tempo_contrato_meses
    ,last(INFOS_MEMBROS.status) as status

    ,max(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as max_tempo_sami_meses
    ,min(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as min_tempo_sami_meses
    ,avg(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as media_tempo_sami_meses

    --- motivo mais frequente

    ----------------- dados demográficos
    ,last(estados_freq.estado) as estado_moda
    ,last(estados_freq.freq) as freq_estado
    ,count(distinct estados_freq.estado) as qtd_estados

    ,sum(INFOS_MEMBROS.genero_mas) as qtd_homens
    ,sum(INFOS_MEMBROS.genero_fem) as qtd_mulheres

    ,max(INFOS_MEMBROS.age) as idade_max
    ,min(INFOS_MEMBROS.age) as idade_min
    ,avg(INFOS_MEMBROS.age) as idade_media
    ,sum(INFOS_MEMBROS.idoso) as qtd_idosos
    ,sum(INFOS_MEMBROS.jovem) as qtd_jovens
    ,sum(INFOS_MEMBROS.adulto) as qtd_adultos
    ,sum(INFOS_MEMBROS.crianca) as qtd_criancas

    ------------ gympass
    ,max(GYMPASS.qtd_dias_gympass) as max_qtd_dias_gympass
    ,avg(GYMPASS.qtd_dias_gympass) as media_qtd_dias_gympass
    ,min(GYMPASS.qtd_dias_gympass) as min_qtd_dias_gympass

    ----------- dados de inadimplência
    ,sum(total_dias_atraso) as soma_dias_atraso_inad
    ,mean(total_dias_atraso) as media_dias_atraso_inad
    ,sum(total_vl_mensalidade) as soma_vl_mensalidade_inad
    ,avg(total_vl_mensalidade) as media_vl_mensalidade_inad

    --- variáveis indicadoras (inadimplemente , cancelados)

    ------------------- tickets
    --- qtd tickets, qtd tickets rel cancelamento, tempo medio, max, min de atendimento
    ,count(tiquete) as qtd_tickets
    ,sum(relacionado_a_cancelamento) as qtd_tickets_cancelamento
    ,sum(relacionado_a_cancelamento)/count(tiquete) as pct_tickets_cancelamento
    ,avg(t_atend_hora) as media_t_atend_hora
    ,max(t_atend_hora) as max_t_atend_hora
    ,min(t_atend_hora) as min_t_atend_hora
    ,avg(t_atend_s) as media_t_atend_s
    ,max(t_atend_s) as max_t_atend_s
    ,min(t_atend_s) as min_t_atend_s

    ------------------- NPS
    --- nota max, med, min
    --- qtd/pct de promotores e detratores, proporção detratores/promotores
    ---- tipo do contato mais frequente
    --- pesquisa mais frequente
    ,avg(nps_agg.nota_nps_media) as nota_media_nps
    ,max(nps_agg.nota_nps_max) as nota_max_nps
    ,min(nps_agg.nota_nps_min) as nota_min_nps
    ,count(nps_agg.qtd_notas_nps_boas) as qtd_notas_boas_nps
    ,count(nps_agg.qtd_notas_nps_medias) as qtd_notas_medias_nps
    ,count(nps_agg.qtd_notas_nps_ruins) as qtd_notas_baixas_nps
    ,last(nps_agg.tipo_do_contato_nps) as tipo_contato_nps

    -------------------- guias
    -- media de guias, soma de guias ,
    ---- qtd meses,
    -----qtd guias aceitas, qtd guias negadas, pct guias aceitas, proporção negadas/aceitas
    --- prestador
    ,avg(USO_GUIAS.media_guias_por_mes) as qtd_media_guias_por_mes
    ,sum(USO_GUIAS.soma_qtd_guias) as soma_qtd_guias
    ,avg(USO_GUIAS.pct_guias_autorizadas) as media_pct_guias_autorizadas
    ,max(USO_GUIAS.pct_guias_autorizadas) as max_pct_guias_autorizadas
    ,min(USO_GUIAS.pct_guias_autorizadas) as min_pct_guias_autorizadas
    ,last(prestadores.prestador_mais_freq) as prestador_mais_freq

    --------------------------- DS
    --- nota max media e min
    ,avg(DS.decl_saude_nota_para_saude) as nota_DS_media
    ,max(DS.decl_saude_nota_para_saude) as nota_DS_max
    ,min(DS.decl_saude_nota_para_saude) as nota_DS_min

    ---------------------- HRA
    ----- fuma? -> qtd fumantes, pct fumantes
    ----- pratica ativ fisica, pct pratica ativ fisica
    ----- nota da saúde -- nota media, min, max
    ----- IMC -> a partir da altura e peso , IMC max, med, min, pct pessoas obesas,pct acima do peso, pct abaixo do peso ideal
    ,avg(hra_agg.numa_escala_de_1_a_10_qual_nota_voce_daria_para_a_sua_saude_hoje) as nota_saude_HRA_media
    ,sum(
      case when hra_agg.voce_fuma = 'yes' then 1
           when hra_agg.voce_fuma = 'no' then 0
           else null end
    ) as qtd_fumantes
    ,sum(
      case when hra_agg.voce_fuma = 'yes' then 1
           when hra_agg.voce_fuma = 'no' then 0
           else null end
    )/count(INFOS_MEMBROS.member_id) as pct_fumantes
    ,sum(
      case when hra_agg.faz_atividade_fisica_regularmente = 'yes' then 1
           when hra_agg.faz_atividade_fisica_regularmente = 'no' then 0
           else null end
    ) as qtd_praticam_atividade
    ,sum(
      case when hra_agg.faz_atividade_fisica_regularmente = 'yes' then 1
           when hra_agg.faz_atividade_fisica_regularmente = 'no' then 0
           else null end
    )/count(INFOS_MEMBROS.member_id) as pct_praticam_atividade

    ,sum(
      case when hra_agg.avaliacao_imc in ('sobrepeso', 'obeso', 'muito obeso') then 1
            else 0 end
    ) as qtd_sobrepeso
    ,sum(
      case when hra_agg.avaliacao_imc in ('sobrepeso', 'obeso', 'muito obeso') then 1
            else 0 end
    )/count(INFOS_MEMBROS.member_id) as pct_sobrepeso

    ---------- aquisição

    ,last(aquisicao.origem_do_lead) as fonte_lead
    ,last(aquisicao.dt_pgto_agendado) as ultima_dt_pgto_agendado

    ---- principal fonte, campanha
    ---- participou indica sami? (sim ou nao, ou qtd)
    ---- fluxo B2B ou B2C
    ---- dt pagamento agendado (primeira, ultima)
    ---- valor da campanha?

    ------------------------- score de crédito
    ----- score medio, max, min
    ---- qtd score alto, qtd score baixo, pct score alto, pct score baixo
    ----- tipo da empresa? ramo da empresa? MEI/eirelli/...?

    --------------------- sinistro
    ----


from INFOS_MEMBROS

left join RELATORIO81_CANCELAMENTO on
  RELATORIO81_CANCELAMENTO.cd_contrato = INFOS_MEMBROS.contract_id

-- left join desligamento on
--   desligamento.cd_contrato = INFOS_MEMBROS.contract_id

left join membros_desligados on
--   membros_desligados.contract_id = INFOS_MEMBROS.contract_id
  membros_desligados.member_id = INFOS_MEMBROS.member_id

left join estados_freq on
--   estados_freq.contract_id = INFOS_MEMBROS.contract_id
  estados_freq.member_id = INFOS_MEMBROS.member_id

left join GYMPASS on
  GYMPASS.health_card_number = INFOS_MEMBROS.health_card_number

left join PERFIL_INAD on
  PERFIL_INAD.cd_contrato = INFOS_MEMBROS.contract_id

left join TICKETS on
  TICKETS.cpf = INFOS_MEMBROS.identity_number

-- left join retorno_nps_agg on
--   retorno_nps_agg.cpf = INFOS_MEMBROS.identity_number
left join nps_agg on
  nps_agg.cpf = INFOS_MEMBROS.identity_number

left join USO_GUIAS on
    USO_GUIAS.nr_carteira_utilizada = INFOS_MEMBROS.health_card_number

left join prestadores on
  prestadores.contract_id = INFOS_MEMBROS.contract_id

left join DS on
  DS.cpf = INFOS_MEMBROS.identity_number

left join hra_agg on
  hra_agg.responsedby = INFOS_MEMBROS.identity_number

left join aquisicao on
  aquisicao.contract_id = INFOS_MEMBROS.contract_id
where INFOS_MEMBROS.contract_id in (479, 480, 647, 136, 127, 1476, 1477)

group by INFOS_MEMBROS.contract_id
order by INFOS_MEMBROS.contract_id

In [ ]:
%sql
WITH INFOS_MEMBROS AS (
  select distinct

    membros.contract_id
    ,membros.member_id
    ,membros.product_id
    ,membros.company_id
    ,membros.health_card_number
    ,membros.identity_number
    ,titular_id

    --- infos gerais
--     ,datediff(getdate(), membros.member_entrance_date) as tempo_de_casa
    ,membros.member_entrance_date
    ,membros.gender
    ,membros.age
    ,membros.status
    ,membros.neighborhood
    ,membros.city
    ,membros.state
    ,membros.dependency_relationship

    --- estatísticas de idades
    ,max(membros.age) over (partition by contract_id) as idade_max
    ,min(membros.age) over (partition by contract_id) as idade_min

    --- dependentes
    ,count(membros.dependency_relationship) over (partition by contract_id) as qtd_dependentes -- se for titular é null

    --- entrada
    ,membros.member_entrance_date

     --- variáveis indicadoras
    ,case when membros.gender = 'M' then 1 else 0 end as genero_mas
    ,case when membros.gender = 'F' then 1 else 0 end as genero_fem

    ,case when membros.age >= 60 then 1 else 0 end as idoso
    ,case when membros.age > 18 and membros.age <= 30 then 1 else 0 end as jovem
    ,case when membros.age > 30 and membros.age < 60 then 1 else 0 end as adulto
    ,case when membros.age <= 18 then 1 else 0 end as crianca


  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  where is_samer = 'nao' -- sem colaboradores
  order by 1
)
,estados_freq as (
  select
    INFOS_MEMBROS.contract_id
    ,INFOS_MEMBROS.member_id
    ,INFOS_MEMBROS.state as estado
    , count(INFOS_MEMBROS.state) as freq
  from INFOS_MEMBROS
  group by
            INFOS_MEMBROS.contract_id
            ,INFOS_MEMBROS.member_id
            , INFOS_MEMBROS.state
  order by
      INFOS_MEMBROS.member_id
      ,count(INFOS_MEMBROS.state)
      desc
)
,RELATORIO81_CANCELAMENTO AS (
   select distinct
     cd_contrato as cd_contrato,
     max(tempo_de_sami_meses) over (partition by cd_contrato) as tempo_de_sami_meses -- maior tempo de contrato por contrato (tempo de contrato mais atualizado)
  from relatorio81_cancel
  order by 1
)
,desligamento as (
  select
    cd_contrato
    ,1 as desligado
    ,dt_desligamento
  from hive_metastore.samidp_prd_refined_oracle_dbaps.desliga_contrato desl
  where dt_reativacao is null
)
,membros_desligados as (
  select
    membros.contract_id
    ,membros.member_id
    ,desligamento.dt_desligamento
    ,COALESCE(desligamento.desligado, 0) AS e_desligado
    ,datediff(desligamento.dt_desligamento, membros.member_entrance_date)/30 as tempo_churn_meses

    ,case when desligamento.dt_desligamento is not null then
              datediff(desligamento.dt_desligamento, membros.member_entrance_date)/30
          when desligamento.dt_desligamento is null and membros.status ='ativo' then
              datediff(CURRENT_TIMESTAMP(), membros.member_entrance_date)/30
          else null
          end as tempo_contrato_meses

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros

  left join desligamento on
    desligamento.cd_contrato = membros.contract_id

--   group by membros.contract_id
  order by 1
)
,GYMPASS AS (
  select
    numero_da_carteirinha as health_card_number
    ,count(data_de_utilizacao_gympass_date) as qtd_dias_gympass
    ,sum(contagens_de_utilizadores_gympass) as qtd_utilizadores_gympass
  from gympass
  group by numero_da_carteirinha
  order by 1
)
,PERFIL_INAD AS (
  select
    cd_contrato
    ,sum(dias_atraso) as total_dias_atraso
    ,avg(dias_atraso) as media_dias_atraso
    ,sum(vl_mensalidade) as total_vl_mensalidade
    ,avg(vl_mensalidade) as media_vl_mensalidade
    -- motivo

  from relatorio81_inad
  group by cd_contrato
  order by 1
)
,TICKETS AS (
  select
    contatos_hs.cpf
    ,tickets_agg.*
  from tickets_agg
  inner join contatos_hs on
  tickets_agg.id_cont = contatos_hs.id
)
, nps_agg as (
  select
      retorno_nps_agg.cpf
      ,avg(retorno_nps_agg.nota) as nota_nps_media
      ,max(retorno_nps_agg.nota) as nota_nps_max
      ,min(retorno_nps_agg.nota) as nota_nps_min
      ,sum(
        case when retorno_nps_agg.nota >= 7 then 1
            else 0 end
      ) as qtd_notas_nps_boas
      ,sum(
        case when retorno_nps_agg.nota > 5 and  retorno_nps_agg.nota < 7 then 1
            else 0 end
      ) as qtd_notas_nps_medias
      ,sum(
        case when retorno_nps_agg.nota <= 5 then 1
            else 0 end
      ) as qtd_notas_nps_ruins
      , last(retorno_nps_agg.tipo_do_contato) as tipo_do_contato_nps
      ,retorno_nps_agg.tipo_do_contato
      ,count(retorno_nps_agg.tipo_do_contato) as freq_tipo_do_contato_nps
  from retorno_nps_agg
  where retorno_nps_agg.cpf is not null
  group by retorno_nps_agg.cpf
          , retorno_nps_agg.tipo_do_contato
  order by retorno_nps_agg.cpf
          , retorno_nps_agg.tipo_do_contato desc
)
,USO_GUIAS as (
  with uso_mensal as (
    select distinct
      nr_carteira_utilizada
      ,DATE_FORMAT(dt_autorizacao, 'yyyyMM') as anomes
      ,count(*) over (partition by nr_carteira_utilizada, DATE_FORMAT(dt_autorizacao, 'yyyyMM')) as qtd_guias_mes
      ,count(DATE_FORMAT(dt_autorizacao, 'yyyyMM')) over (partition by nr_carteira_utilizada) as qtd_meses
      ,count(*) over (partition by nr_carteira_utilizada) as qtd_guias
      ,case when dt_autorizacao is not null ---and ds_senha_autorizacao is not null
          then 1 else 0
          end as autorizacao
    from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
    where dt_autorizacao is not null
    order by 1,2
  )
  select distinct
    uso_mensal.nr_carteira_utilizada
    ,avg(uso_mensal.qtd_guias_mes) as media_guias_por_mes
    ,avg(uso_mensal.qtd_guias) as soma_qtd_guias
    ,count(distinct uso_mensal.anomes) as qtd_meses
--     ,avg(uso_mensal.qtd_guias) / count(distinct uso_mensal.anomes) as razao_qtd_guias_qtd_meses
    ,sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes) as qtd_guias_autorizadas
    ,(sum(uso_mensal.autorizacao)/count(distinct uso_mensal.anomes))/avg(uso_mensal.qtd_guias) as pct_guias_autorizadas

  from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
  left join uso_mensal on
    uso_mensal.nr_carteira_utilizada = guia.nr_carteira_utilizada

  where dt_autorizacao is not null
  group by uso_mensal.nr_carteira_utilizada
  order by 1
)
,prestadores as (
  with prestadores_freq as (
    select distinct
      membros.contract_id
      ,guia.cd_prestador
      ,count(guia.cd_prestador) as freq_prestador

    from hive_metastore.samidp_prd_refined_oracle_dbaps.guia guia
    left join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
      guia.nr_carteira_utilizada = membros.health_card_number
    group by
      membros.contract_id
      ,guia.cd_prestador
    order by 3 asc
  )
  select distinct
    membros.contract_id
    ,first(prestadores_freq.freq_prestador) over (
          partition by prestadores_freq.contract_id order by prestadores_freq.freq_prestador desc
          ) as freq_prestador_mais_freq
    ,first(prestadores_freq.cd_prestador) over (
          partition by prestadores_freq.contract_id order by prestadores_freq.freq_prestador desc
          ) as prestador_mais_freq

  from hive_metastore.samidp_prd_refined_core_datasets.dim_member membros
  left join prestadores_freq on
    membros.contract_id = prestadores_freq.contract_id
  order by 1
)
,DS as (
  select
      responsedby as cpf
      ,answers_value as decl_saude_nota_para_saude
  from declaracao_saude
)
,aquisicao as (
  select distinct
    membros.contract_id
    ,aquisicao.origem_do_lead
    ,aquisicao.dt_pgto_agendado
    ,aquisicao.dt_create

  from hive_metastore.samidp_prd_refined_pme_portal_pme.companies c
  inner join aquisicao on
    aquisicao.cnpj = c.document
  inner join hive_metastore.samidp_prd_refined_core_datasets.dim_member membros on
    membros.contract_id = c.customerid
  order by membros.contract_id, aquisicao.dt_pgto_agendado
)
select
--     distinct
    INFOS_MEMBROS.contract_id
    ,count(INFOS_MEMBROS.member_id) as qtd_membros

    ---------------- dados de cancelamento
   ,last(e_desligado) AS e_desligado
   ,last(tempo_churn_meses) as tempo_churn_meses
    ,last(tempo_contrato_meses) as tempo_contrato_meses
    ,last(INFOS_MEMBROS.status) as status

    ,max(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as max_tempo_sami_meses
    ,min(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as min_tempo_sami_meses
    ,avg(RELATORIO81_CANCELAMENTO.tempo_de_sami_meses) as media_tempo_sami_meses

    --- motivo mais frequente

    ----------------- dados demográficos
    ,last(estados_freq.estado) as estado_moda
    ,last(estados_freq.freq) as freq_estado
    ,count(distinct estados_freq.estado) as qtd_estados

    ,sum(INFOS_MEMBROS.genero_mas) as qtd_homens
    ,sum(INFOS_MEMBROS.genero_fem) as qtd_mulheres

    ,max(INFOS_MEMBROS.age) as idade_max
    ,min(INFOS_MEMBROS.age) as idade_min
    ,avg(INFOS_MEMBROS.age) as idade_media
    ,sum(INFOS_MEMBROS.idoso) as qtd_idosos
    ,sum(INFOS_MEMBROS.jovem) as qtd_jovens
    ,sum(INFOS_MEMBROS.adulto) as qtd_adultos
    ,sum(INFOS_MEMBROS.crianca) as qtd_criancas

    ------------ gympass
    ,max(GYMPASS.qtd_dias_gympass) as max_qtd_dias_gympass
    ,avg(GYMPASS.qtd_dias_gympass) as media_qtd_dias_gympass
    ,min(GYMPASS.qtd_dias_gympass) as min_qtd_dias_gympass

    ----------- dados de inadimplência
    ,sum(total_dias_atraso) as soma_dias_atraso_inad
    ,mean(total_dias_atraso) as media_dias_atraso_inad
    ,sum(total_vl_mensalidade) as soma_vl_mensalidade_inad
    ,avg(total_vl_mensalidade) as media_vl_mensalidade_inad

    --- variáveis indicadoras (inadimplemente , cancelados)

    ------------------- tickets
    --- qtd tickets, qtd tickets rel cancelamento, tempo medio, max, min de atendimento
    ,count(tiquete) as qtd_tickets
    ,sum(relacionado_a_cancelamento) as qtd_tickets_cancelamento
    ,sum(relacionado_a_cancelamento)/count(tiquete) as pct_tickets_cancelamento
    ,avg(t_atend_hora) as media_t_atend_hora
    ,max(t_atend_hora) as max_t_atend_hora
    ,min(t_atend_hora) as min_t_atend_hora
    ,avg(t_atend_s) as media_t_atend_s
    ,max(t_atend_s) as max_t_atend_s
    ,min(t_atend_s) as min_t_atend_s

    ------------------- NPS
    --- nota max, med, min
    --- qtd/pct de promotores e detratores, proporção detratores/promotores
    ---- tipo do contato mais frequente
    --- pesquisa mais frequente
    ,avg(nps_agg.nota_nps_media) as nota_media_nps
    ,max(nps_agg.nota_nps_max) as nota_max_nps
    ,min(nps_agg.nota_nps_min) as nota_min_nps
    ,count(nps_agg.qtd_notas_nps_boas) as qtd_notas_boas_nps
    ,count(nps_agg.qtd_notas_nps_medias) as qtd_notas_medias_nps
    ,count(nps_agg.qtd_notas_nps_ruins) as qtd_notas_baixas_nps
    ,last(nps_agg.tipo_do_contato_nps) as tipo_contato_nps

    -------------------- guias
    -- media de guias, soma de guias ,
    ---- qtd meses,
    -----qtd guias aceitas, qtd guias negadas, pct guias aceitas, proporção negadas/aceitas
    --- prestador
    ,avg(USO_GUIAS.media_guias_por_mes) as qtd_media_guias_por_mes
    ,sum(USO_GUIAS.soma_qtd_guias) as soma_qtd_guias
    ,avg(USO_GUIAS.pct_guias_autorizadas) as media_pct_guias_autorizadas
    ,max(USO_GUIAS.pct_guias_autorizadas) as max_pct_guias_autorizadas
    ,min(USO_GUIAS.pct_guias_autorizadas) as min_pct_guias_autorizadas
    ,last(prestadores.prestador_mais_freq) as prestador_mais_freq

    --------------------------- DS
    --- nota max media e min
    ,avg(DS.decl_saude_nota_para_saude) as nota_DS_media
    ,max(DS.decl_saude_nota_para_saude) as nota_DS_max
    ,min(DS.decl_saude_nota_para_saude) as nota_DS_min

    ---------------------- HRA
    ----- fuma? -> qtd fumantes, pct fumantes
    ----- pratica ativ fisica, pct pratica ativ fisica
    ----- nota da saúde -- nota media, min, max
    ----- IMC -> a partir da altura e peso , IMC max, med, min, pct pessoas obesas,pct acima do peso, pct abaixo do peso ideal
    ,avg(hra_agg.numa_escala_de_1_a_10_qual_nota_voce_daria_para_a_sua_saude_hoje) as nota_saude_HRA_media
    ,sum(
      case when hra_agg.voce_fuma = 'yes' then 1
           when hra_agg.voce_fuma = 'no' then 0
           else null end
    ) as qtd_fumantes
    ,sum(
      case when hra_agg.voce_fuma = 'yes' then 1
           when hra_agg.voce_fuma = 'no' then 0
           else null end
    )/count(INFOS_MEMBROS.member_id) as pct_fumantes
    ,sum(
      case when hra_agg.faz_atividade_fisica_regularmente = 'yes' then 1
           when hra_agg.faz_atividade_fisica_regularmente = 'no' then 0
           else null end
    ) as qtd_praticam_atividade
    ,sum(
      case when hra_agg.faz_atividade_fisica_regularmente = 'yes' then 1
           when hra_agg.faz_atividade_fisica_regularmente = 'no' then 0
           else null end
    )/count(INFOS_MEMBROS.member_id) as pct_praticam_atividade

    ,sum(
      case when hra_agg.avaliacao_imc in ('sobrepeso', 'obeso', 'muito obeso') then 1
            else 0 end
    ) as qtd_sobrepeso
    ,sum(
      case when hra_agg.avaliacao_imc in ('sobrepeso', 'obeso', 'muito obeso') then 1
            else 0 end
    )/count(INFOS_MEMBROS.member_id) as pct_sobrepeso

    ---------- aquisição

    ,last(aquisicao.origem_do_lead) as fonte_lead
    ,last(aquisicao.dt_pgto_agendado) as ultima_dt_pgto_agendado

    ---- principal fonte, campanha
    ---- participou indica sami? (sim ou nao, ou qtd)
    ---- fluxo B2B ou B2C
    ---- dt pagamento agendado (primeira, ultima)
    ---- valor da campanha?

    ------------------------- score de crédito
    ----- score medio, max, min
    ---- qtd score alto, qtd score baixo, pct score alto, pct score baixo
    ----- tipo da empresa? ramo da empresa? MEI/eirelli/...?

    --------------------- sinistro
    ----


from INFOS_MEMBROS

left join RELATORIO81_CANCELAMENTO on
  RELATORIO81_CANCELAMENTO.cd_contrato = INFOS_MEMBROS.contract_id

-- left join desligamento on
--   desligamento.cd_contrato = INFOS_MEMBROS.contract_id

left join membros_desligados on
--   membros_desligados.contract_id = INFOS_MEMBROS.contract_id
  membros_desligados.member_id = INFOS_MEMBROS.member_id

left join estados_freq on
--   estados_freq.contract_id = INFOS_MEMBROS.contract_id
  estados_freq.member_id = INFOS_MEMBROS.member_id

left join GYMPASS on
  GYMPASS.health_card_number = INFOS_MEMBROS.health_card_number

left join PERFIL_INAD on
  PERFIL_INAD.cd_contrato = INFOS_MEMBROS.contract_id

left join TICKETS on
  TICKETS.cpf = INFOS_MEMBROS.identity_number

-- left join retorno_nps_agg on
--   retorno_nps_agg.cpf = INFOS_MEMBROS.identity_number
left join nps_agg on
  nps_agg.cpf = INFOS_MEMBROS.identity_number

left join USO_GUIAS on
    USO_GUIAS.nr_carteira_utilizada = INFOS_MEMBROS.health_card_number

left join prestadores on
  prestadores.contract_id = INFOS_MEMBROS.contract_id

left join DS on
  DS.cpf = INFOS_MEMBROS.identity_number

left join hra_agg on
  hra_agg.responsedby = INFOS_MEMBROS.identity_number

left join aquisicao on
  aquisicao.contract_id = INFOS_MEMBROS.contract_id
where INFOS_MEMBROS.contract_id in (479, 480, 647, 136, 127, 1476, 1477)

group by INFOS_MEMBROS.contract_id
order by INFOS_MEMBROS.contract_id